# SUUMO データ可視化

このノートブックでは、スクレイピングしたSUUMOデータの可視化を行います。

## 目次
1. ライブラリの準備とデータ読み込み
2. 賃料分布の可視化
3. 駅別分析の可視化
4. 面積と賃料の関係
5. 間取り分布
6. 築年数の影響
7. ヒートマップ分析
8. 総合ダッシュボード

In [ ]:
# 必要なライブラリのインポート
import sys
import os
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

from analyzer import SuumoAnalyzer
from visualizer import SuumoVisualizer

# 日本語フォント設定
plt.rcParams['font.family'] = ['DejaVu Sans', 'Hiragino Sans', 'Yu Gothic', 'Meiryo']
sns.set_style("whitegrid")
sns.set_palette("husl")

# Plotlyの設定
import plotly.io as pio
pio.renderers.default = "notebook"

print("📊 SUUMO データ可視化ノートブック")
print("=================================\n")

## 1. ライブラリの準備とデータ読み込み

In [ ]:
# データの読み込み
print("🔄 データを読み込み中...")
analyzer = SuumoAnalyzer(data_path="../../data", auto_load=True)

if analyzer.df is not None:
    print(f"✅ データ読み込み完了: {len(analyzer.df)} 件")
    
    # 可視化器の初期化
    visualizer = SuumoVisualizer(analyzer.df, output_dir="../visualizations")
    print("🎨 可視化器の初期化完了")
else:
    print("❌ データの読み込みに失敗しました")
    raise ValueError("データが読み込めませんでした")

In [ ]:
# データ概要の確認
print("📋 データ概要:")
print(f"   行数: {len(analyzer.df):,}")
print(f"   列数: {len(analyzer.df.columns)}")

if 'search_station' in analyzer.df.columns:
    stations = analyzer.df['search_station'].unique()
    print(f"   対象駅: {', '.join(stations)}")

# 使用可能な数値カラムの確認
numeric_cols = analyzer.df.select_dtypes(include=[np.number]).columns.tolist()
print(f"   数値カラム: {', '.join(numeric_cols)}")

## 2. 賃料分布の可視化

In [ ]:
# 全体の賃料分布
print("💰 賃料分布ヒストグラム")
fig1 = visualizer.plot_rent_distribution(by_station=False, save=False)
fig1.show()

In [ ]:
# 駅別賃料分布
if 'search_station' in analyzer.df.columns:
    print("🚉 駅別賃料分布")
    fig2 = visualizer.plot_rent_distribution(by_station=True, save=False)
    fig2.show()
else:
    print("⚠️ 駅情報が利用できません")

## 3. 駅別分析の可視化

In [ ]:
# 駅別賃料ボックスプロット
if 'search_station' in analyzer.df.columns and 'rent_numeric' in analyzer.df.columns:
    print("📊 駅別賃料ボックスプロット")
    fig3 = visualizer.plot_rent_by_station(save=False)
    fig3.show()
else:
    print("⚠️ 駅別賃料分析に必要なデータが不足しています")

In [ ]:
# 駅別平米単価比較
if 'search_station' in analyzer.df.columns and 'rent_per_sqm' in analyzer.df.columns:
    print("💹 駅別平米単価比較")
    fig4 = visualizer.plot_rent_per_sqm_by_station(save=False)
    fig4.show()
else:
    print("⚠️ 平米単価データが利用できません")

## 4. 面積と賃料の関係

In [ ]:
# 面積と賃料の散布図（間取り別）
if 'area_numeric' in analyzer.df.columns and 'rent_numeric' in analyzer.df.columns:
    print("📐 面積と賃料の関係（間取り別）")
    fig5 = visualizer.plot_area_vs_rent(color_by='layout', save=False)
    fig5.show()
else:
    print("⚠️ 面積または賃料データが利用できません")

In [ ]:
# 面積と賃料の散布図（駅別）
if 'area_numeric' in analyzer.df.columns and 'rent_numeric' in analyzer.df.columns and 'search_station' in analyzer.df.columns:
    print("🚉 面積と賃料の関係（駅別）")
    fig6 = visualizer.plot_area_vs_rent(color_by='search_station', save=False)
    fig6.show()
else:
    print("⚠️ 必要なデータが利用できません")

## 5. 間取り分布

In [ ]:
# 間取り分布円グラフ
if 'layout' in analyzer.df.columns:
    print("🏠 間取り分布")
    fig7 = visualizer.plot_layout_distribution(save=False)
    fig7.show()
    
    # 間取り別統計も表示
    print("\n📊 間取り別統計:")
    layout_stats = analyzer.df.groupby('layout').agg({
        'rent_numeric': ['count', 'mean', 'median'],
        'area_numeric': 'mean'
    }).round(2)
    
    layout_stats.columns = ['件数', '平均賃料', '中央値賃料', '平均面積']
    layout_stats = layout_stats.sort_values('件数', ascending=False)
    display(layout_stats)
else:
    print("⚠️ 間取りデータが利用できません")

## 6. 築年数の影響

In [ ]:
# 築年数と賃料の関係
if 'building_age' in analyzer.df.columns and 'rent_numeric' in analyzer.df.columns:
    print("🏗️ 築年数と賃料の関係")
    fig8 = visualizer.plot_age_vs_rent(save=False)
    fig8.show()
else:
    print("⚠️ 築年数または賃料データが利用できません")

In [ ]:
# 築年数カテゴリ別分析
if 'age_category' in analyzer.df.columns:
    print("📊 築年数カテゴリ別分析")
    
    # カテゴリ別ボックスプロット
    fig_age = px.box(
        analyzer.df,
        x='age_category',
        y='rent_numeric',
        title='築年数カテゴリ別賃料分布',
        labels={'age_category': '築年数カテゴリ', 'rent_numeric': '賃料 (円)'}
    )
    fig_age.update_layout(title_x=0.5, xaxis_tickangle=-45)
    fig_age.show()
else:
    print("⚠️ 築年数カテゴリが作成されていません")

## 7. ヒートマップ分析

In [ ]:
# 駅×間取りヒートマップ
if 'search_station' in analyzer.df.columns and 'layout' in analyzer.df.columns:
    print("🔥 駅×間取り物件数ヒートマップ")
    fig9 = visualizer.plot_heatmap_station_layout(save=False)
    fig9.show()
else:
    print("⚠️ 駅または間取りデータが利用できません")

In [ ]:
# 価格帯分布（駅別スタックドバー）
if 'search_station' in analyzer.df.columns and 'rent_range' in analyzer.df.columns:
    print("💰 駅別価格帯分布")
    fig10 = visualizer.plot_price_range_distribution(save=False)
    fig10.show()
else:
    print("⚠️ 価格帯データが利用できません")

## 8. 相関分析

In [ ]:
# 数値変数間の相関分析
numeric_cols = ['rent_numeric', 'area_numeric', 'building_age', 'rent_per_sqm']
available_cols = [col for col in numeric_cols if col in analyzer.df.columns]

if len(available_cols) >= 2:
    print("🔗 数値変数間の相関分析")
    
    # 相関マトリックス計算
    corr_matrix = analyzer.df[available_cols].corr()
    
    # ヒートマップ作成
    fig_corr = px.imshow(
        corr_matrix,
        title='変数間相関マトリックス',
        color_continuous_scale='RdBu',
        aspect='auto',
        text_auto='.2f'
    )
    fig_corr.update_layout(title_x=0.5)
    fig_corr.show()
    
    print("\n📊 相関係数マトリックス:")
    display(corr_matrix.round(3))
else:
    print("⚠️ 相関分析に必要な数値データが不足しています")

## 9. 総合ダッシュボード

In [ ]:
# 総合ダッシュボード
print("🎯 総合ダッシュボード")
fig11 = visualizer.plot_comprehensive_dashboard(save=False)
fig11.show()

## 10. カスタム分析

In [ ]:
# お得物件の可視化
if 'rent_per_sqm' in analyzer.df.columns and 'area_numeric' in analyzer.df.columns:
    print("💎 お得物件分析")
    
    # お得度スコアを計算（簡易版）
    df_copy = analyzer.df.copy()
    
    # 正規化スコア計算
    df_copy['area_score'] = df_copy['area_numeric'] / df_copy['area_numeric'].mean()
    df_copy['price_score'] = 1 - (df_copy['rent_per_sqm'] / df_copy['rent_per_sqm'].mean())
    
    if 'building_age' in df_copy.columns:
        df_copy['age_score'] = 1 - (df_copy['building_age'] / df_copy['building_age'].max())
        df_copy['deal_score'] = (df_copy['area_score'] * 0.4 + 
                                df_copy['price_score'] * 0.4 + 
                                df_copy['age_score'] * 0.2)
    else:
        df_copy['deal_score'] = (df_copy['area_score'] * 0.5 + 
                                df_copy['price_score'] * 0.5)
    
    # 散布図でお得度を可視化
    fig_deal = px.scatter(
        df_copy,
        x='area_numeric',
        y='rent_numeric',
        color='deal_score',
        size='deal_score',
        title='お得物件分析（面積 vs 賃料 + お得度スコア）',
        labels={'area_numeric': '面積 (㎡)', 'rent_numeric': '賃料 (円)', 'deal_score': 'お得度スコア'},
        color_continuous_scale='Viridis',
        hover_data=['building_title'] if 'building_title' in df_copy.columns else None
    )
    fig_deal.update_layout(title_x=0.5)
    fig_deal.show()
    
    # TOP10お得物件
    top_deals = df_copy.nlargest(10, 'deal_score')
    print("\n🏆 お得度TOP10:")
    
    display_cols = ['building_title', 'search_station', 'rent_numeric', 'area_numeric', 'deal_score']
    available_display_cols = [col for col in display_cols if col in top_deals.columns]
    
    if available_display_cols:
        display(top_deals[available_display_cols].round(3))
else:
    print("⚠️ お得物件分析に必要なデータが不足しています")

## 11. 全ての可視化を保存

In [ ]:
# 全ての可視化をファイルに保存
print("💾 全ての可視化を保存中...")
try:
    generated_files = visualizer.generate_all_visualizations()
    
    print("\n✅ 保存完了:")
    for key, description in generated_files.items():
        print(f"   📊 {description}")
    
    print(f"\n📂 保存先: {visualizer.output_dir}")
    print("\n💡 生成されたHTMLファイルをブラウザで開いてインタラクティブな可視化をお楽しみください！")
    
except Exception as e:
    print(f"❌ 保存中にエラーが発生しました: {e}")

## まとめ

この可視化ノートブックでは、以下の分析を行いました：

### 📊 実行した可視化
1. **賃料分布**: 全体および駅別の賃料ヒストグラム
2. **駅別分析**: ボックスプロットと平米単価比較
3. **面積vs賃料**: 間取り別・駅別の散布図
4. **間取り分布**: 円グラフと統計表
5. **築年数影響**: 築年数と賃料の関係性
6. **ヒートマップ**: 駅×間取りの物件数分布
7. **相関分析**: 数値変数間の相関マトリックス
8. **総合ダッシュボード**: 複数の可視化を統合
9. **お得物件分析**: スコアベースの物件評価

### 🎯 次のステップ
- **特定駅の詳細分析**: `03_station_comparison.ipynb`
- **予測モデル構築**: `04_price_prediction.ipynb`
- **地理的分析**: 地図上での物件分布（将来の拡張）

### 💾 出力ファイル
全ての可視化は `analysis/visualizations/` ディレクトリに保存されています。HTMLファイルはブラウザで開いてインタラクティブに操作できます。